In [7]:
import geopandas as gpd
import pandas as pd
import contextily as cx
import numpy as np
from pointpats import centrography
from shapely.geometry import Point
import matplotlib.pyplot as plt
from scipy import stats
import shapely as shapely
import seaborn as sns

In [ ]:
%%R
library(readr)
library(data.table)
library(tidyverse)
library(dplyr)
library(sp)
library(sf)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
stations_full <- read_sf("stations_full.geojson")

stations_UTID <- stations_full %>% filter(grepl("US1ID|US1UT", ID))

stations_WYCO <- stations_full %>% filter(grepl("US1CO|US1WY", ID))

st_write(stations_UTID, "stations_UTID.geojson")
st_write(stations_WYCO, "stations_WYCO.geojson")

In [2]:
stations_full = gpd.read_file(r"C:\Users\p7j5p\Documents\Paul's reports\spatial statistics\indv files\stations_full.geojson")

In [3]:
stations_full

,ID,year,month,element,Date,Amt (mm),Lat,Long,geometry
0,US1COAD0001,2002,1,SNOW,Val1,13.0,39.946,-104.6088,POINT (-10093313.994 4701775.544)
1,US1COAD0001,2002,1,SNOW,Val10,104.0,39.946,-104.6088,POINT (-10093313.994 4701775.544)
2,US1COAD0001,2002,2,SNOW,Val25,38.0,39.946,-104.6088,POINT (-10093313.994 4701775.544)
3,US1COAD0001,2002,2,SNOW,Val26,13.0,39.946,-104.6088,POINT (-10093313.994 4701775.544)
4,US1COAD0001,2002,3,SNOW,Val1,89.0,39.946,-104.6088,POINT (-10093313.994 4701775.544)
...,...,...,...,...,...,...,...,...,...
267821,US1WYWS0029,2020,3,SNOW,Val28,13.0,43.7019,-104.6947,POINT (-10101602.165 5060908.463)
267822,US1WYWS0029,2020,4,SNOW,Val2,64.0,43.7019,-104.6947,POINT (-10101602.165 5060908.463)
267823,US1WYWS0029,2020,4,SNOW,Val12,64.0,43.7019,-104.6947,POINT (-10101602.165 5060908.463)
267824,US1WYWS0030,2018,1,SNOW,Val14,71.0,44.1263,-104.3947,POINT (-10072656.281 5100163.709)


In [5]:
print(stations_full.crs)

epsg:6933


In [12]:
stations_UTID = gpd.read_file(r"C:\Users\p7j5p\Documents\Paul's reports\spatial statistics\indv files\stations_UTID.geojson")
stations_WYCO = gpd.read_file(r"C:\Users\p7j5p\Documents\Paul's reports\spatial statistics\indv files\stations_WYCO.geojson")

In [31]:
stations_UTID['month'] = pd.to_numeric(stations_UTID['month'], errors='coerce')
stations_WYCO['month'] = pd.to_numeric(stations_WYCO['month'], errors='coerce')

In [26]:
stations_UTID.dtypes

ID            object
year          object
month          int64
element       object
Date          object
Amt..mm.     float64
Lat           object
Long          object
geometry    geometry
dtype: object

subsetting the data to test the hypothesis

In [32]:
stations_UTID12 = stations_UTID.loc[(stations_UTID['month'] < 3)]
stations_UTID34 = stations_UTID.loc[(stations_UTID['month'] > 2)]
stations_WYCO12 = stations_WYCO.loc[(stations_WYCO['month'] < 3)]
stations_WYCO34 = stations_WYCO.loc[(stations_WYCO['month'] > 2)]

In [30]:
stations_UTID34

,ID,year,month,element,Date,Amt..mm.,Lat,Long,geometry
9,US1IDAD0002,2009,3,SNOW,Val5,18.0,43.6695,-116.2653,POINT (-11218006.319 5057900.273)
10,US1IDAD0002,2009,3,SNOW,Val6,20.0,43.6695,-116.2653,POINT (-11218006.319 5057900.273)
11,US1IDAD0002,2009,3,SNOW,Val9,33.0,43.6695,-116.2653,POINT (-11218006.319 5057900.273)
14,US1IDAD0002,2010,3,SNOW,Val13,33.0,43.6695,-116.2653,POINT (-11218006.319 5057900.273)
21,US1IDAD0002,2011,3,SNOW,Val17,8.0,43.6695,-116.2653,POINT (-11218006.319 5057900.273)
...,...,...,...,...,...,...,...,...,...
19344,US1UTWY0001,2009,3,SNOW,Val10,25.0,38.3943,-111.5911,POINT (-10767010.148 4547487.624)
19345,US1UTWY0001,2009,4,SNOW,Val4,76.0,38.3943,-111.5911,POINT (-10767010.148 4547487.624)
19346,US1UTWY0001,2009,4,SNOW,Val11,25.0,38.3943,-111.5911,POINT (-10767010.148 4547487.624)
19354,US1UTWY0001,2010,3,SNOW,Val7,89.0,38.3943,-111.5911,POINT (-10767010.148 4547487.624)


In [16]:
from scipy import stats

In [34]:
UTID12 = stations_UTID12['Amt..mm.'].values
UTID34 = stations_UTID34['Amt..mm.'].values
WYCO12 = stations_WYCO12['Amt..mm.'].values
WYCO34 = stations_WYCO34['Amt..mm.'].values

In [37]:
stats.mannwhitneyu(UTID12, WYCO12, alternative='greater')

MannwhitneyuResult(statistic=992193232.0, pvalue=0.3422076652274443)

fail to reject null

In [36]:
stats.mannwhitneyu(WYCO34, UTID34, alternative='greater')

MannwhitneyuResult(statistic=322724868.5, pvalue=2.4445271794549023e-43)

yayy significant at the highest level, reject null

In [38]:
stats.mannwhitneyu(UTID12, WYCO12)

MannwhitneyuResult(statistic=992193232.0, pvalue=0.6844153304548886)